In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import leidenalg
import umap

In [ ]:
# transferred cell embeddings
sc_dataset = np.load("scVAE_Embedding.npy")
cell = pd.DataFrame(sc_dataset, index=[f'X{i}' for i in range(sc_dataset.shape[0])])
adata = sc.AnnData(sc_dataset)# 转换为 AnnData 对象
adata.obs_names = ["X"+str(name) for name in adata.obs_names]
cell.to_csv('sc_sfvae.csv', index=False)

In [ ]:
# transferred sample embeddings
bulk_dataset = np.load("bulkVAE_Embedding.npy")
sample = pd.DataFrame(bulk_dataset)
sample.to_csv('bulk_sfVAE.csv', index=False)

In [ ]:
# umap coordinates
reducer = umap.UMAP(n_components=2, random_state=42)
sc_embedding = reducer.fit_transform(sc_dataset)
pd.DataFrame(sc_embedding).to_csv('umap_embedding_sfvae.csv', index=False)

In [ ]:
# Build KNN graph and convert to SNN graph (weighted adjacency graph)
sc.pp.neighbors(adata, n_neighbors=15, use_rep='X')  # 使用高维 embedding 计算 KNN 图

# Apply Leiden clustering
sc.tl.leiden(adata, resolution=0.5)  # 分辨率调整聚类数量

# Leiden cluster labels are stored in adata.obs['leiden']
leiden_labels = adata.obs['leiden'].to_numpy()

leiden_cluster = pd.DataFrame({"Leiden_Cluster": leiden_labels})
leiden_cluster.to_csv("leiden_cluster.csv",index=False)